In [1]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score , StratifiedKFold

import pymystem3
from natasha import NamesExtractor

%matplotlib inline

/home/kirill/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def validate(x , y):
    model = LGBMClassifier(n_estimators=670 ,learning_rate=0.09 , num_leaves=53 , subsample=0.9 , subsample_for_bin=60000 , min_child_samples = 10 , random_state = 42)
    cv = StratifiedKFold(4 ,shuffle=True, random_state=99)
    score = cross_val_score(model , x , y , scoring='roc_auc' , cv=cv)
    return score.mean() , score.std()

### Загружаем данные

In [3]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
all_data = pd.concat([train , test])

sample = pd.read_csv('data/sample_submission.csv')

### Pymystem

In [4]:
%%time
mystem = pymystem3.Mystem()

def name_from_pymystem(word):
    try:
        return 1 if 'имя' in mystem.analyze(word)[0]['analysis'][0]['gr'].split(',') else 0
    except:
        return 0

def surn_from_pymystem(word):
    try:
        return 1 if 'фам' in mystem.analyze(word)[0]['analysis'][0]['gr'].split(',') else 0
    except:
        return 0

all_data['name_from_pymystem'] = all_data['Word'].apply(name_from_pymystem)
all_data['surn_from_pymystem'] = all_data['Word'].apply(surn_from_pymystem)

CPU times: user 8.96 s, sys: 1.68 s, total: 10.6 s
Wall time: 24 s


In [5]:
new_train = all_data[all_data['Label'].notnull()]
validate(new_train.drop(['Label' , 'Word' ] , axis=1) , new_train['Label'])

(0.75716231264334644, 0.0054610382001182012)

### Natasha

In [6]:
%%time
func = NamesExtractor()
def function_natasha(word):
    return 1 if func(word) else 0

all_data['natasha_person'] = all_data['Word'].apply(function_natasha)

CPU times: user 3min 31s, sys: 76 ms, total: 3min 31s
Wall time: 3min 31s


In [7]:
new_train = all_data[all_data['Label'].notnull()]
new_test = all_data[all_data['Label'].isnull()]
validate(new_train.drop(['Label' , 'Word' ] , axis=1) , new_train['Label'])

(0.80039869041500378, 0.0023417885851331633)

In [ ]:
model = XGBClassifier(max_depth = 10 , n_estimators=670 , learning_rate=0.09 , colsample_bytree=0.9 , colsample_bylevel=0.6)
model.fit(new_train.drop(['Label' , 'Word'] , axis=1) , new_train['Label'])
sample['Prediction'] = model.predict_proba(new_test.drop(['Word' , 'Label'] , axis=1))[:,0]
sample.to_csv('submit.csv' , index=False)